<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkBroadcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('SparkBroadcast').getOrCreate()

In [2]:
spark

In [8]:
transactions=[
    (4, 'Apples', 1.03),
    (3, 'Apples', 2.68),
    (4, 'Oranges', 4.72),
    (1, 'Apples', 1.94),
    (3, 'Bread', 3.07),
    (1, 'Apples', 0.75),
    (4, 'Bananas', 4.71),
    (2, 'Bread', 2.86),
    (3, 'Bread', 2.61),
    (3, 'Apples', 0.64),
    (1, 'Oranges', 1.60),
    (1, 'Bread', 1.50)
]

schema=['store_id', 'product', 'price']
df1=spark.createDataFrame(data=transactions, schema=schema)
df1.show()


+--------+-------+-----+
|store_id|product|price|
+--------+-------+-----+
|       4| Apples| 1.03|
|       3| Apples| 2.68|
|       4|Oranges| 4.72|
|       1| Apples| 1.94|
|       3|  Bread| 3.07|
|       1| Apples| 0.75|
|       4|Bananas| 4.71|
|       2|  Bread| 2.86|
|       3|  Bread| 2.61|
|       3| Apples| 0.64|
|       1|Oranges|  1.6|
|       1|  Bread|  1.5|
+--------+-------+-----+



In [9]:
store=[
    (1, "New York"),
    (2, "Los Angeles"),
    (3, "Chicago"),
    (4, "Houston")
]
schema1=['store_id', 'store_name']
df2=spark.createDataFrame(data=store, schema=schema1)
df2.show()

+--------+-----------+
|store_id| store_name|
+--------+-----------+
|       1|   New York|
|       2|Los Angeles|
|       3|    Chicago|
|       4|    Houston|
+--------+-----------+



    Normal Join

In [10]:
df3=df1.join(df2, on='store_id', how='inner')
df3.show()

+--------+-------+-----+-----------+
|store_id|product|price| store_name|
+--------+-------+-----+-----------+
|       1| Apples| 1.94|   New York|
|       1| Apples| 0.75|   New York|
|       1|Oranges|  1.6|   New York|
|       1|  Bread|  1.5|   New York|
|       2|  Bread| 2.86|Los Angeles|
|       3| Apples| 2.68|    Chicago|
|       3|  Bread| 3.07|    Chicago|
|       3|  Bread| 2.61|    Chicago|
|       3| Apples| 0.64|    Chicago|
|       4| Apples| 1.03|    Houston|
|       4|Oranges| 4.72|    Houston|
|       4|Bananas| 4.71|    Houston|
+--------+-------+-----+-----------+



In [12]:
df3.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [store_id])
:- LogicalRDD [store_id#66L, product#67, price#68], false
+- LogicalRDD [store_id#85L, store_name#86], false

== Analyzed Logical Plan ==
store_id: bigint, product: string, price: double, store_name: string
Project [store_id#66L, product#67, price#68, store_name#86]
+- Join Inner, (store_id#66L = store_id#85L)
   :- LogicalRDD [store_id#66L, product#67, price#68], false
   +- LogicalRDD [store_id#85L, store_name#86], false

== Optimized Logical Plan ==
Project [store_id#66L, product#67, price#68, store_name#86]
+- Join Inner, (store_id#66L = store_id#85L)
   :- Filter isnotnull(store_id#66L)
   :  +- LogicalRDD [store_id#66L, product#67, price#68], false
   +- Filter isnotnull(store_id#85L)
      +- LogicalRDD [store_id#85L, store_name#86], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(5) Project [store_id#66L, product#67, price#68, store_name#86]
   +- *(5) SortMergeJoin [store_id#66L

    Using BroadCast join

In [13]:
from pyspark.sql.functions import *
df4=df1.join(broadcast(df2), on='store_id', how='inner')
df4.show()

+--------+-------+-----+-----------+
|store_id|product|price| store_name|
+--------+-------+-----+-----------+
|       4| Apples| 1.03|    Houston|
|       3| Apples| 2.68|    Chicago|
|       4|Oranges| 4.72|    Houston|
|       1| Apples| 1.94|   New York|
|       3|  Bread| 3.07|    Chicago|
|       1| Apples| 0.75|   New York|
|       4|Bananas| 4.71|    Houston|
|       2|  Bread| 2.86|Los Angeles|
|       3|  Bread| 2.61|    Chicago|
|       3| Apples| 0.64|    Chicago|
|       1|Oranges|  1.6|   New York|
|       1|  Bread|  1.5|   New York|
+--------+-------+-----+-----------+



In [14]:
df4.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [store_id])
:- LogicalRDD [store_id#66L, product#67, price#68], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [store_id#85L, store_name#86], false

== Analyzed Logical Plan ==
store_id: bigint, product: string, price: double, store_name: string
Project [store_id#66L, product#67, price#68, store_name#86]
+- Join Inner, (store_id#66L = store_id#85L)
   :- LogicalRDD [store_id#66L, product#67, price#68], false
   +- ResolvedHint (strategy=broadcast)
      +- LogicalRDD [store_id#85L, store_name#86], false

== Optimized Logical Plan ==
Project [store_id#66L, product#67, price#68, store_name#86]
+- Join Inner, (store_id#66L = store_id#85L), rightHint=(strategy=broadcast)
   :- Filter isnotnull(store_id#66L)
   :  +- LogicalRDD [store_id#66L, product#67, price#68], false
   +- Filter isnotnull(store_id#85L)
      +- LogicalRDD [store_id#85L, store_name#86], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project

    We could see in the explain plan, where in Normal join we could see shuffing part.
    
    When we applied broadcast for df2 and joined, we could see no data shuffled.

**END OF CODE**